In [ ]:
################## auto login ######################
import login
from login import *
import pandas as pd
from loguru import logger
import pytz
from time import sleep
import datetime
import json
import pandas as pd
# logger.add(".\logs\MNC_logs_{time}.log")
mypath = 'C:\Users\YOGESH\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\Scripts\trade_scripts_notes'
logger.add(mypath+"/logs/MNC_logs_{time}.log")

In [ ]:
################# Parameters #######################
my_margin = 160000.00
reserved_margin = 10000.00
squareup_percent = 10.00
################## Constants #######################
exchange_type = kite.EXCHANGE_NSE
available_margin_for_trades = my_margin - reserved_margin
n50_token = 256265
n50_symbol = 'NIFTY 50'
days_ago=3
prev_gapup_down_limit = 2.00

In [ ]:
############# Import Old qty_df File ###############
import os.path
from os import path
is_qty_df_available = path.exists('qty_df.p')
old_qty_df = pd.DataFrame()
if(is_qty_df_available):
    old_qty_df = joblib.load('.\qty_df.p')
# old_qty_df

In [ ]:
############### Get Stocks Margin Map ############
import requests
url = 'https://api.kite.trade/margins/equity'
margin_data = requests.get(url, timeout=2.50)
margin_map = json.loads(margin_data.content.decode('utf-8'))
margin_map_df = pd.DataFrame(margin_map)

############ Get N/50 Stocks List ################
n50df = pd.read_excel(mypath+'\n50.xlsx')
n50df = n50df.loc[n50df['tradable'] == 'y']

############### Merge Both Data ##################
merged_df = pd.merge(n50df, margin_map_df, on="tradingsymbol")
merged_df['instrument_token'] = merged_df['instrument_token'].astype(int)
merged_df['exchange_token'] = merged_df['exchange_token'].astype(int)
# merged_df

In [ ]:
############## Stock Margin Calucator for Bracket Order #####################
def calc_margin(co_upper,co_lower,price,stoploss=100, transaction_type = 'sell',quantity=1):
    '''calc_margin(co_upper,co_lower,stk_data,price,stoploss=10, transaction_type = 'sell',quantity=1)'''
    co_lower = co_lower/100
    co_upper = co_upper/100
    trigger = price - (co_upper * price)
    
    if stoploss < trigger:
        stoploss = trigger
    else:
        trigger = stoploss
    x = 0
    if transaction_type == 'buy':
        x = (price - trigger) * quantity
    else:
        x = (trigger - price) * quantity
        
    y = co_lower * price * quantity
#     margin  = x if( x > y) else y
#     margin = margin + (margin * 0.2)
    return y

############# Get 9:15 Data ##############################################
def get_mornign_data( inst_token, interval='5minute'):
    '''get_mornign_data( inst_token, interval='5minute')'''
    interval=interval
    inst_token = inst_token
    todaydt=datetime.date.today()
    hud_ago=todaydt-datetime.timedelta(days=days_ago)
    to_date=datetime.date.isoformat(todaydt)

    from_date=datetime.date.isoformat(hud_ago)
    data = kite.historical_data(inst_token, from_date, to_date, interval, continuous=False)
    prev_close = kite.quote([inst_token])[str(inst_token)]['ohlc']['close']
    mydata = pd.DataFrame(data)
    today = datetime.date.today()
#     print('Today Date : '+str(today))
    yest_date =  mydata[-days_ago:][['date']].to_numpy()[0][0]
    last_close =  mydata[-days_ago:][['close']].to_numpy()[0][0]
    last_high =  mydata[-days_ago:][['high']].to_numpy()[0][0]
    last_low =  mydata[-days_ago:][['low']].to_numpy()[0][0]
#     print('yest_date : '+str(yest_date))
#     print('last close : '+ str(last_close))
#     print('last high : '+ str(last_high))
#     print('last low : '+ str(last_low))

    import pytz
    
    tz = pytz.timezone("Asia\Calcutta")
    
    this_day = today.day
    now_time = datetime.datetime.now()
    
    if(now_time.hour <8 ):
        last_day = today-datetime.timedelta(days=1)
        this_day = last_day.day
    
    b =tz.localize( datetime.datetime(today.year, today.month, this_day, 9, 15,0,0))
    # today.day
    timestamp = pd.Timestamp(b)
    mycandle = mydata.loc[mydata['date'] == timestamp]
    
    buy_exit = mycandle['high'].to_numpy()[0]*((100.0+squareup_percent)/100.0)
    sell_exit = mycandle['low'].to_numpy()[0]*((100.0-squareup_percent)/100.0)
    
    buy_exit = round(0.05 * round(buy_exit /0.05),2)
    sell_exit = round(0.05 * round(sell_exit /0.05),2)
    
    return {'open':mycandle['open'].to_numpy()[0],
            'high':mycandle['high'].to_numpy()[0],
            'low':mycandle['low'].to_numpy()[0],
            'close':mycandle['close'].to_numpy()[0],
            'prev_close':prev_close,
            'buy_exit':buy_exit,
            'sell_exit':sell_exit,}

########### Decide Buy/Sell ###############
def decide_buy_sell(morning_candle):
    direction = ''
    if(morning_candle['open']>morning_candle['close']):
        direction = 'sell'
    if(morning_candle['open']<morning_candle['close']):
        direction = 'buy'
    return direction

In [ ]:
# ################ Wait Till 9:20 #####################
# def act(x):
#     return x+10

# def wait_start(runTime, action):
#     startTime = datetime.time(*(map(int, runTime.split(':'))))
#     while startTime > datetime.datetime.today().time(): 
#         sleep(1)
#     return action

# wait_start('9:20', lambda: act(100))

In [ ]:
################# Add Buy/Sell Direction to merged_df ############
direction_list = []
for i in merged_df.instrument_token.to_list():
    this_map = get_mornign_data(i)
    this_dir = decide_buy_sell(this_map)
    direction_list.append({'instrument_token':i,'direction':this_dir,
                           'morning_open':this_map['open'],
                           'morning_high':this_map['high'],
                           'morning_low':this_map['low'],
                           'morning_close':this_map['close'],
                           'prev_close':this_map['prev_close'],
                           'buy_exit':this_map['buy_exit'],
                           'sell_exit':this_map['sell_exit'],})
    print(i)

    
direction_list_df = pd.DataFrame(direction_list)
merged_df2 = pd.merge(merged_df, direction_list_df, on="instrument_token")
# merged_df2

In [ ]:
################ Get Considered DF Comparing to N50 Direction ########## 
nifty50_morning_candle = get_mornign_data(n50_token)
nifty_direction = decide_buy_sell(nifty50_morning_candle)
considered_df = merged_df2.loc[merged_df2['direction'] == nifty_direction]
# considered_df
nifty_direction

In [ ]:
################### Decide Qty for Every Symbol #####################
qty_df_list = []
for inst in considered_df.instrument_token:
    this_co_upper = considered_df.loc[considered_df['instrument_token'] ==
                                      inst]['co_upper'].to_numpy()[0]
    this_co_lower = considered_df.loc[considered_df['instrument_token'] ==
                                      inst]['co_lower'].to_numpy()[0]
    this_price = 1.0
    this_sl = 1.0
    if (nifty_direction == 'buy'):
        this_price = considered_df.loc[considered_df['instrument_token'] ==
                                       inst]['morning_high'].to_numpy()[0]
        this_sl = considered_df.loc[considered_df['instrument_token'] ==
                                    inst]['morning_low'].to_numpy()[0]
    else:
        this_price = considered_df.loc[considered_df['instrument_token'] ==
                                       inst]['morning_low'].to_numpy()[0]
        this_sl = considered_df.loc[considered_df['instrument_token'] ==
                                    inst]['morning_high'].to_numpy()[0]
    this_marg = calc_margin(this_co_upper,
                            this_co_lower,
                            this_price,
                            stoploss=this_sl,
                            transaction_type=nifty_direction)
    remaining_weight = 100-considered_df.sum()['weight']
    remaining_weight_per_symbol = remaining_weight/(len(considered_df.instrument_token.to_list()))
    this_weight = considered_df.loc[considered_df['instrument_token'] == inst]['weight'].to_numpy()[0]
#     surplus_weight = this_weight * (remaining_weight/100)
    this_net_weight = this_weight + remaining_weight_per_symbol
    weighted_margin = available_margin_for_trades * this_net_weight/100
    this_qty = int(weighted_margin / this_marg)
    this_gap_upper_limit = (considered_df.loc[considered_df['instrument_token'] ==
                                       inst]['prev_close'].to_numpy()[0])*((100+prev_gapup_down_limit)/100.00)
    
    this_gap_lower_limit = (considered_df.loc[considered_df['instrument_token'] ==
                                       inst]['prev_close'].to_numpy()[0])*((100-prev_gapup_down_limit)/100.00)
    
    
    this_gap_upper_limit = round(0.05 * round(this_gap_upper_limit /0.05),2)
    this_gap_lower_limit = round(0.05 * round(this_gap_lower_limit /0.05),2)
    
    qty_df_list.append({
        'instrument_token': inst,
        'co_bo_margin': this_marg,
        'weighted_margin':weighted_margin,
        'trade_qty': this_qty,
        'new_weight':this_net_weight,
        'my_qty': 0,
        'ltp': 0.0,
        'stop_trading': False,
        'last_updated_on': str(datetime.date.today()),
        'order_id':111111,
        'gap_upper_limit':this_gap_upper_limit,
        'gap_lower_limit':this_gap_lower_limit
    })

qty_df = pd.DataFrame(qty_df_list)
qty_df = pd.merge(considered_df, qty_df, on="instrument_token")
if(not old_qty_df.empty):
    qty_df['stop_trading'] = old_qty_df['stop_trading']
    qty_df['last_updated_on'] = old_qty_df['last_updated_on']
    qty_df['order_id'] = old_qty_df['order_id']

qty_df

In [ ]:
###################### Clear Yesterday True/False ##################################
for my_inst in qty_df.instrument_token:
    this_last_up = str(qty_df.loc[qty_df['instrument_token'] ==
                                                         my_inst]['last_updated_on'].to_numpy()[0]).split('-')[2]
    this_last_up = int(this_last_up)
    this_day = datetime.date.today().day
    if(this_day!=this_last_up):
        qty_df.at[qty_df.index[qty_df['instrument_token'] == my_inst].tolist()[0],'stop_trading']= False
        qty_df.at[qty_df.index[qty_df['instrument_token'] == my_inst].tolist()[0],'last_updated_on']= str(datetime.date.today())
        qty_df.at[qty_df.index[qty_df['order_id'] == 111111
        print('Stop Trading Changed for  : '+str(my_inst))

joblib.dump(qty_df,mypath+'\qty_df.p')

In [ ]:
qty_df.to_excel('.\qty_df_updated.xlsx')

In [ ]:
############################## Get My Qty #########################
myquantity = 0
def getquant():
    global myquantity
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        for j in qty_df.instrument_token.to_list():
            if(allpos[i]['instrument_token']==j and  allpos[i]['product']== 'CO'):
                myquantity = allpos[i]['quantity']
                qty_df.at[qty_df.index[qty_df['instrument_token'] == j].tolist()[0],'my_qty']= myquantity
                this_smb = qty_df.loc[qty_df['instrument_token'] == j]['tradingsymbol'].to_numpy()[0]
                print('My Quantity : '+ this_smb + ' : ' + str(allpos[i]['quantity']))
    joblib.dump(qty_df,mypath+'\qty_df.p')
getquant()
########################### Place Order ###########################
placing_order = False
def placeneworder(inst_token,quantdiff,trigger_price):
    '''placeneworder(inst_token,quantdiff,trigger_price)'''
    global stopbuy,stopsell,placing_order
    my_symbol_ip = qty_df.loc[qty_df['instrument_token'] == inst_token]['tradingsymbol'].to_numpy()[0]
        
    if(quantdiff>0):
        placing_order = True
        try:
            order_id= kite.place_order(tradingsymbol=my_symbol_ip, 
                        exchange=kite.EXCHANGE_NSE, 
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        trigger_price = trigger_price,
                        quantity=abs(quantdiff),
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_CO,
                        product=kite.PRODUCT_CO,)
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'stop_trading']= True
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'last_updated_on']= datetime.date.today()
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'order_id']=order_id
            getquant()
            joblib.dump(qty_df,mypath+'\qty_df.p')
        except Exception as e:
            print(e)
            getquant()
        placing_order = False
    if(quantdiff<0):
        placing_order = True
        try:
            order_id= kite.place_order(tradingsymbol=my_symbol_ip, 
                    exchange=kite.EXCHANGE_NSE, 
                    transaction_type=kite.TRANSACTION_TYPE_SELL,
                    trigger_price = trigger_price,
                    quantity=abs(quantdiff),
                    order_type=kite.ORDER_TYPE_MARKET,
                    variety = kite.VARIETY_CO,
                    product=kite.PRODUCT_CO,)
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'stop_trading']= True
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'last_updated_on']= datetime.date.today()
            qty_df.at[qty_df.index[qty_df['instrument_token'] == inst_token].tolist()[0],'order_id']=order_id
            getquant()
            joblib.dump(qty_df,mypath+'\qty_df.p')
            
        except Exception as e:
            print(e)
            getquant()
        placing_order = False
############### CHK QTY #####################################
def ckqnt (tkn,orderquant,trigger_price):
    '''ckqnt (tkn,orderquant,trigger_price)'''
    global placing_order
    myquantity=qty_df.loc[qty_df['instrument_token'] == tkn]['my_qty'].to_numpy()[0]
    quantdiff = orderquant - myquantity
    if(placing_order==False and quantdiff!=0):
        placeneworder(inst_token=tkn,quantdiff=quantdiff,trigger_price=trigger_price)
    return myquantity

In [ ]:
# from pivottablejs import pivot_ui
# from IPython.display import HTML
# pvi = pivot_ui(qty_df,outfile_path='./pivottablejs.html')
# pvi

In [ ]:
old_qty_df.to_excel(mypath+'\old_qty_df.xlsx')
qty_df.to_excel(mypath+'\qty_df.xlsx')

In [ ]:
####################### Start Trading #########################
import logging
import threading
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)

counter = 0

def on_ticks(ws, ticks):
    global counter
    counter = counter + 1
    print('==========================')
    this_min = datetime.datetime.now().minute
    this_hour = datetime.datetime.now().hour

    for one_stk_data in ticks:
        print('Symbol : ' + str(qty_df.loc[qty_df['instrument_token'] ==
                                           one_stk_data['instrument_token']]
                                ['tradingsymbol'].to_numpy()[0]))
        print('LTP : ' + str(one_stk_data['last_price']))

        this_ltp = one_stk_data['last_price']

        this_morning_low = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['morning_low'].to_numpy()[0]
        this_morning_high = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['morning_high'].to_numpy()[0]
        this_stk_qty = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['trade_qty'].to_numpy()[0]
        this_stop_trading = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['stop_trading'].to_numpy()[0]
        last_ltp = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['ltp'].to_numpy()[0]
        this_my_qty = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['my_qty'].to_numpy()[0]
        this_order_id = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['order_id'].to_numpy()[0]
        this_buy_exit = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['buy_exit'].to_numpy()[0]
        this_sell_exit = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['sell_exit'].to_numpy()[0]

        this_gap_upper_limit1 = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['gap_upper_limit'].to_numpy()[0]
        this_gap_lower_limit1 = qty_df.loc[
            qty_df['instrument_token'] ==
            one_stk_data['instrument_token']]['gap_lower_limit'].to_numpy()[0]

        qty_df.at[qty_df.index[qty_df['instrument_token'] ==
                               one_stk_data['instrument_token']].tolist()[0],
                  'ltp'] = this_ltp

        if (nifty_direction == 'sell'):
#             print('Morning Low : ' + str(this_morning_low))
        else:
#             print('Morning High : ' + str(this_morning_high))
#         print('*****************************')

        is_9_25_up = this_hour == 9 and this_min <= 25

        gap_condition = this_morning_low > this_gap_lower_limit1 and this_morning_high < this_gap_upper_limit1

        if (this_hour <= 10 and this_min <= 45 and this_stk_qty != 0
                and this_stop_trading == False):
            
            if (nifty_direction == 'sell' and this_ltp < this_morning_low
                    and (is_9_25_up or last_ltp >= this_morning_low)
                    and gap_condition):
                ckqnt(one_stk_data['instrument_token'], -this_stk_qty,
                      this_morning_high)
            
            if (nifty_direction == 'buy' and this_ltp > this_morning_high
                    and (is_9_25_up or last_ltp <= this_morning_high)
                    and gap_condition):
                ckqnt(one_stk_data['instrument_token'], this_stk_qty,
                      this_morning_low)

        if (this_my_qty > 0):
            x = threading.Thread(target=getquant, args=())
            if (nifty_direction == 'sell' and this_ltp <= this_sell_exit):
                kite.cancel_order(kite.VARIETY_CO, this_order_id)
                x.start()
            if (nifty_direction == 'buy' and this_ltp >= this_buy_exit):
                kite.cancel_order(kite.VARIETY_CO, this_order_id)
                x.start()

    if (counter % 10 == 0):
        print(counter)
        qty_df.to_csv(mypath+'\live_data.csv')
        joblib.dump(qty_df, mypath+'\qty_df.p')

    print('==========================')
    print()


def on_connect(ws, response):
    ws.subscribe(qty_df.instrument_token.to_list())
    ws.set_mode(ws.MODE_FULL, qty_df.instrument_token.to_list())

def on_close(ws, code, reason):
    ws.stop()

def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

def on_order_update(ws, data):
    print("order update: ", data)

kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

In [ ]:
# kite.place_order(
#     tradingsymbol = 'ONGC',
#     transaction_type =kite.TRANSACTION_TYPE_BUY,
#     trigger_price = 70.00,
#     quantity = 1,
#     order_type=kite.ORDER_TYPE_MARKET,
#     variety=kite.VARIETY_CO,
#     exchange=kite.EXCHANGE_NSE,
#     product=kite.PRODUCT_CO)

In [ ]:
# inst = kite.instruments()
# leninst = len(inst)
# n50list = []
# for i in range(leninst):
#     for j in range(len(nifty50)):
#         if(inst[i]['tradingsymbol']==nifty50[j] and inst[i]['exchange']=='NSE'):
#             n50list.append(inst[i])
# n50list
# mydf = pd.DataFrame(n50list)  
# mydf.to_csv('n50.csv')  